In [ ]:
import pandas as pd

from datetime import datetime
from sklearn import datasets, ensemble, model_selection

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import ProbClassificationPerformanceTab

from evidently.model_profile import Profile
from evidently.model_profile.sections import ProbClassificationPerformanceProfileSection

# Breast Cancer Data

In [ ]:
bcancer = datasets.load_breast_cancer()

In [ ]:
bcancer_frame = pd.DataFrame(bcancer.data, columns = bcancer.feature_names)

In [ ]:
bcancer_frame['target'] = bcancer.target

In [ ]:
bcancer_frame.head()

In [ ]:
target = 'target'
prediction = 'prediction'

numerical_features = bcancer.feature_names
categorical_features = []

features = numerical_features.tolist() + categorical_features

# Probabilistic Model Performance Dashboard

In [ ]:
train_data, test_data = model_selection.train_test_split(bcancer_frame, random_state=0)

In [ ]:
model = ensemble.RandomForestClassifier(random_state=0)

In [ ]:
model.fit(train_data[features], train_data.target)

In [ ]:
# make a data frame with predictions for each class

train_probas = pd.DataFrame(model.predict_proba(train_data[features]))
train_probas.columns = bcancer.target_names
test_probas = pd.DataFrame(model.predict_proba(test_data[features]))
test_probas.columns = bcancer.target_names

In [ ]:
train_probas.head()

In [ ]:
# get labels for target: [0, 1, 0, 2] -> ['setosa', 'versicolor', 'setosa', 'virginica']

train_data['target'] = [bcancer.target_names[x] for x in train_data['target']]
test_data['target'] = [bcancer.target_names[x] for x in test_data['target']]

In [ ]:
# merge train and test data with predictions

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

merged_train_data = pd.concat([train_data, train_probas], axis=1)
merged_test_data = pd.concat([test_data, test_probas], axis=1)

In [ ]:
merged_train_data.head()

In [ ]:
bcancer_column_mapping = ColumnMapping()

bcancer_column_mapping.target = target
bcancer_column_mapping.prediction = bcancer.target_names.tolist()
bcancer_column_mapping.numerical_features = numerical_features

## Probabilistic Model Performance Dashboard full (verbose_level=1)

In [ ]:
bcancer_model_performance_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=1)])
bcancer_model_performance_dashboard.calculate(merged_train_data, merged_test_data, 
                                              column_mapping = bcancer_column_mapping)
bcancer_model_performance_dashboard.show()

In [ ]:
# bcancer_model_performance_dashboard.save('bcancer_model_performance.html')

## Probabilistic Model Performance Dashboard short (verbose_level=0)

In [ ]:
bcancer_model_performance_dashboard = Dashboard(tabs=[ProbClassificationPerformanceTab(verbose_level=0)])
bcancer_model_performance_dashboard.calculate(merged_train_data, merged_test_data, 
                                              column_mapping = bcancer_column_mapping)
bcancer_model_performance_dashboard.show()

In [ ]:
# bcancer_model_performance_dashboard.save('bcancer_model_performance.html')

# Model Performance Profile

In [ ]:
bcancer_classification_performance_profile = Profile(sections=[ProbClassificationPerformanceProfileSection()])

In [ ]:
bcancer_classification_performance_profile.calculate(merged_train_data, merged_test_data, 
                                                     column_mapping = bcancer_column_mapping)

In [ ]:
bcancer_classification_performance_profile.json() 